In [27]:
# imports
import pyodbc
import pandas as pd
from datetime import date, datetime

In [2]:
# digitenis database connection
dsn = 'sqlserverdatasource'
user = 'sa'
password = 'digiTenis1@'
database = 'digitenis'

conn_string = 'DSN=%s;UID=%s;PWD=%s;DATABASE=%s;' % (dsn, user, password, database)
conn = pyodbc.connect(conn_string)

In [ ]:
# dataframe tables
fornecedor = pd.read_sql('select * from fornecedor', conn)
telefone = pd.read_sql('select * from telefone', conn)
funcionario = pd.read_sql('select * from funcionario', conn)
produto = pd.read_sql('select * from produto', conn)
venda = pd.read_sql('select * from venda', conn)
item_venda = pd.read_sql('select * from item_venda', conn)
estoque = pd.read_sql('select * from estoque', conn)

## Functional Requirements

In [11]:
# cadastrar novos fornecedores
fornecedor.loc[len(fornecedor)] = [21, '6478912300015', 'Olympikus', 'contato@olympikus.com.br']

In [13]:
# cadastrar novos produtos
produto.loc[len(produto)] = [21, 'Anyway 2', 39, 'Corrida', 80, 119.90, 21]

In [14]:
# cadastrar novos funcionários
funcionario.loc[len(funcionario)] = [19, 'Gisele', 'Oliveira', '45695135726', 'Vendedor']

In [28]:
# efetivar vendas
venda.loc[len(venda)] = [29, 359.70, date.today(), datetime.now().time(), 19]

item_venda.loc[len(item_venda)] = [21, 29, 3, 35.97]


In [32]:
# visualizar informações de contato dos fornecedores
fornecedor[['nome', 'email']]

,nome,email
0,Nike,contato@nike.com.br
1,Adidas,contato@adidas.com.br
2,Puma,contato@puma.com.br
3,Mizuno,contato@mizuno.com.br
4,Asics,contato@asics.com.br
5,Under Armour,contato@underarmour.com.br
6,New Balance,contato@newbalance.com.br
7,Skechers,contato@skechers.com.br
8,Reebok,contato@reebok.com.br
9,Fila,contato@fila.com.br


In [43]:
# atualizar valor de venda dos produtos
produto.loc[produto['id_produto'] == 21, 'valor_venda'] = 323.73

In [ ]:
# visualizar estoque por produto
